In [1]:
from matplotlib import pyplot as plt
from matplotlib.dates import DateFormatter
import numpy as np
import xarray as xr
import pandas as pd
import scipy
from datetime import datetime, timedelta
import time
import seaborn
import matplotlib.dates as mdates
import bottleneck as bn
import PyCO2SYS as pyco2
import gsw
import math
import netCDF4 as nc
import requests

from importlib import reload
import warnings
warnings.filterwarnings('ignore')



# Set up API credentials

API credentials are in a different folder so I have to change the path

In [2]:
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

scope = [
   'https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
#Name of our Service Account Key
google_key_file = '../phyto/servicecredentials.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name(google_key_file, scope)
gc = gspread.authorize(credentials)

# URL retrieval

The website format changed so the most up-to-date version of this data retrieval uses `'https://www.pmel.noaa.gov/co2/timeseries/' + dat_list + '.txt'`. Redoing the data retrieval using this new/different website did not yield more sites with pCO2 and pH. However, it did provide measurements for more time steps.

In [3]:
datalist = ['ALAWAI_w_meta.txt',
            'BOBOA_w_meta.txt',
            'BTM_w_meta.txt',
            'CAPEARAGO_w_meta.txt',
            'CAPEELIZABETH_w_meta.txt',
            'CCE1_w_meta.txt',
            'CCE2_w_meta.txt',
            'CHABA_w_meta.txt',
            'CHEECAROCKS_w_meta.txt',
            'CHUUK_w_meta.txt',
            'COASTALLA_w_meta.txt',
            'COASTALMS_w_meta.txt',
            'CRESCENTREEF_w_meta.txt',
            'CRIMP1_w_meta.txt',
            'CRIMP2_w_meta.txt',
            'DABOB_w_meta.txt',
            'FIRSTLANDING_w_meta.txt',
            'GAKOA_w_meta.txt',
            'GRAYSREEF_w_meta.txt',
            'GULFOFMAINE_w_meta.txt',
            'HOGREEF_w_meta.txt',
            'ICELAND_w_meta.txt',
            'JKEO_w_meta.txt',
            'KANEOHE_w_meta.txt',
            'KEO_w_meta.txt',
            'KILONALU_w_meta.txt',
            'KODIAK_w_meta.txt',
            'LAPARGUERA_w_meta.txt',
            'M2_w_meta.txt',
            'NH10_w_meta.txt',
            'PAPA_w_meta.txt',
            'SEAK_w_meta.txt',
            'SOFS_w_meta.txt',
            'STRATUS_w_meta.txt',
            'TAO110W_w_meta.txt',
            'TAO125W_w_meta.txt',
            'TAO140W_w_meta.txt',
            'TAO155W_w_meta.txt',
            'TAO165E_w_meta.txt',
            'TAO170W_w_meta.txt',
            'TAO8S165E_w_meta.txt',
            'TWANOH_w_meta.txt',
            'WHOTS_w_meta.txt']

In [9]:
dat_list = ['CCE1',
            'PAPA',
            'KEO',
            'WHOTS',
            'TAO110W',
            'TAO125W',
            'TAO140W',
            'TAO170W',
            'TAO8S165E',
            'STRATUS',
            'BTM',
            'ICELAND',
            'BOBOA',
            'GAKOA',
            'KODIAK',
            'SEAK',
            'M2',
            'CAPEELIZABETH',
            'CHABA',
            'CCE2',
            'NH10',
            'ALAWAI',
            'CHUUK',
            'CRIMP1',
            'CRIMP2',
            'KANEOHE',
            'KILONALU',
            'GRAYSREEF',
            'GULFOFMAINE',
            'CRESCENTREEF',
            'HOGREEF',
            'COASTALMS',
            'CHEECAROCKS',
            'LAPARGUERA']

In [19]:
# urlbase = 'https://data.pmel.noaa.gov/pmel/erddap/files/all_pmel_co2_moorings/'
urlbase = 'https://www.pmel.noaa.gov/co2/timeseries/'

https://www.ncei.noaa.gov/access/ocean-carbon-acidification-data-system/oceans/Moorings/ndp097.html

1. Get url of tab delimited .txt file of the data. It will be of the form 'https://www.pmel.noaa.gov/co2/timeseries/CCE1.txt'.
2. Skip preamble in the first 109 rows to grab data
3. Main data does not include a column for site name or coordinates. Grab lines 6 and 7 of the preamble for name and location info.
4. Line 6 (now line 0 of a different data object) contains site name. Parse the string to remove '# Site Name: ' and just get the site name. Add this value to the `site` column.
5. Line 7 (now line 1 of diffeerent data object) contains the coordinates as a string. Parse the string to isolate the numeric coordinates. If coordinates are south or west, make them negative.
6. Do this for each site in `dat_list`.

In [158]:
for i in dat_list:
    
    # Create a 'base' dataframe using the first site in the list
    # Subsequent sites will be combined with this dataframe
    if i == 'CCE1':
        # print('CCE1')
        URL = urlbase + i + '.txt'
        df = pd.read_csv(URL, skiprows=109, delimiter='\t')
        
        # Read site name and coords from tsv
        # EXAMPLE
        # NOAA Surface Ocean CO2 and Ocean Acidification Mooring Time Series:
        # 0                     # Site Name: La Parguera                   
        # 1                     # Location:  17.954N,  67.051W                   

        d = pd.read_csv(URL, delimiter='\t', skiprows=5, nrows=2)
        
        # Isolate site name and put in dataframe
        # '# _ S I T E _ N A M E : ' -- site name begins at char 13
        # EXAMPLE: 'La Parguera'
        site = d['# NOAA Surface Ocean CO2 and Ocean Acidification Mooring Time Series:'][0][13:]
        df['site'] = site
        
        # Isolate coords and put in data frame
        # '# Location:  17.954N,  67.051W'
        # '# _ L O C A T I O N: _ ' (13 characters before coords start) 
        # char [13-18): 17.95
        # sig figs provided across sites vary so 3rd digit after decimal dropped
        lat = float(d['# NOAA Surface Ocean CO2 and Ocean Acidification Mooring Time Series:'][1][13:18])
        
        # Since sig figs vary, check both 18th and 19th char for direction
        # If latitude is south, make latitude negative
        if d['# NOAA Surface Ocean CO2 and Ocean Acidification Mooring Time Series:'][1][19] == 'S' or d['# NOAA Surface Ocean CO2 and Ocean Acidification Mooring Time Series:'][1][18] == 'S':
            lat = -lat
        df['latitude'] = lat
        
        # Use length of the string to help parse longitude string
        # '# Location:  17.954N,  67.051W'
        # ' _ 6 7 . 0 5 1' -- long
        leng = len(d['# NOAA Surface Ocean CO2 and Ocean Acidification Mooring Time Series:'][1])
        lon = float(d['# NOAA Surface Ocean CO2 and Ocean Acidification Mooring Time Series:'][1][leng - 8: leng - 1])
        
        # West or east given at leng - 1
        # if west, make longitude negative
        lonWE = d['# NOAA Surface Ocean CO2 and Ocean Acidification Mooring Time Series:'][1][leng - 1: leng]
        if lonWE == 'W':
            lon = -lon
        df['longitude'] = lon
        # df['longdir'] = lonWE
        
    # Add sites to the base dataframe
    else:
        # print('other')
        URL = urlbase + i + '.txt'
        df2 = pd.read_csv(URL, skiprows=109, delimiter='\t')
        
        # Read site name and coords from tsv
        # EXAMPLE
        # NOAA Surface Ocean CO2 and Ocean Acidification Mooring Time Series:
        # 0                     # Site Name: La Parguera                   
        # 1                     # Location:  17.954N,  67.051W   
        
        d = pd.read_csv(URL, delimiter='\t', skiprows=5, nrows=2)
        
        # Isolate site name and put in dataframe
        # '# _ S I T E _ N A M E : ' -- site name begins at char 13
        # EXAMPLE: 'La Parguera'
        site = d['# NOAA Surface Ocean CO2 and Ocean Acidification Mooring Time Series:'][0][13:]
        df2['site'] = site
        
        # Isolate coords and put in data frame
        # '# Location:  17.954N,  67.051W'
        # '# _ L O C A T I O N: _ ' (13 characters before coords start) 
        # char [13-18): 17.95
        # sig figs provided across sites vary so 3rd digit after decimal dropped
        lat = float(d['# NOAA Surface Ocean CO2 and Ocean Acidification Mooring Time Series:'][1][13:18])
        
        # Since sig figs vary, check both 18th and 19th char for direction
        # If latitude is south, make latitude negative
        if d['# NOAA Surface Ocean CO2 and Ocean Acidification Mooring Time Series:'][1][19] == 'S' or d['# NOAA Surface Ocean CO2 and Ocean Acidification Mooring Time Series:'][1][18] == 'S':
            lat = -lat
        df2['latitude'] = lat
        
        # Use length of the string to help parse longitude string
        # '# Location:  17.954N,  67.051W'
        # ' _ 6 7 . 0 5 1' -- long
        leng = len(d['# NOAA Surface Ocean CO2 and Ocean Acidification Mooring Time Series:'][1])
        lon = float(d['# NOAA Surface Ocean CO2 and Ocean Acidification Mooring Time Series:'][1][leng - 8: leng - 1])
        
        # West or east given at leng - 1
        # if west, make longitude negative
        lonWE = d['# NOAA Surface Ocean CO2 and Ocean Acidification Mooring Time Series:'][1][leng - 1: leng]
        if lonWE == 'W':
            lon = -lon
        df2['longitude'] = lon
        # df2['longdir'] = lonWE
        
        # Combine this with preexisting dataframe
        df = pd.concat([df, df2])

# Raw data, no dropped data

As we can see, a lot of values for pH are `Nan`. Some values for pCO2 and a few for SSS are `Nan`.

In [171]:
df

,datetime_utc,SST,SSS,pCO2_sw,pCO2_air,xCO2_air,pH_sw,DOXY,CHL,NTU,site,latitude,longitude
0,2008-11-11 01:29,NaN,NaN,NaN,NaN,386.8,NaN,NaN,NaN,NaN,CCE1,33.48,-122.510
1,2008-11-11 01:59,NaN,NaN,NaN,NaN,386.7,NaN,NaN,NaN,NaN,CCE1,33.48,-122.510
2,2008-11-11 02:29,NaN,NaN,NaN,NaN,386.7,NaN,NaN,NaN,NaN,CCE1,33.48,-122.510
3,2008-11-11 02:59,NaN,NaN,NaN,NaN,386.7,NaN,NaN,NaN,NaN,CCE1,33.48,-122.510
4,2008-11-11 03:29,15.251,33.321,348.1,382.2,386.6,NaN,NaN,NaN,NaN,CCE1,33.48,-122.510
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35826,2021-10-22 21:17,29.985,33.325,472.9,390.8,407.1,NaN,NaN,NaN,NaN,La Parguera,17.95,-67.051
35827,2021-10-23 00:17,29.902,33.310,472.8,396.3,412.1,NaN,NaN,NaN,NaN,La Parguera,17.95,-67.051
35828,2021-10-23 03:17,29.876,33.335,477.9,396.1,411.8,NaN,NaN,NaN,NaN,La Parguera,17.95,-67.051
35829,2021-10-23 06:17,29.819,33.340,488.3,397.3,413.6,NaN,NaN,NaN,NaN,La Parguera,17.95,-67.051


# Isolate data with only `pCO2_sw` and no `pH_sw`

This data will be downloaded and run through LIAR.m to calculate total alkalinity.

In [176]:
df_LIAR = df[df['pH_sw'].isna()]
df_LIAR = df_LIAR[df_LIAR['pCO2_sw'].notna()]
df_LIAR = df_LIAR[df_LIAR['SSS'].notna()]
df_LIAR = df_LIAR.reset_index(drop=True)
df_LIAR

,datetime_utc,SST,SSS,pCO2_sw,pCO2_air,xCO2_air,pH_sw,DOXY,CHL,NTU,site,latitude,longitude
0,2008-11-11 03:29,15.251,33.321,348.1,382.2,386.6,NaN,NaN,NaN,NaN,CCE1,33.48,-122.510
1,2008-11-11 06:17,15.207,33.326,351.0,382.1,386.2,NaN,NaN,NaN,NaN,CCE1,33.48,-122.510
2,2008-11-11 09:17,15.127,33.334,351.6,381.7,385.5,NaN,NaN,NaN,NaN,CCE1,33.48,-122.510
3,2008-11-11 12:17,15.075,33.350,351.7,382.4,386.3,NaN,NaN,NaN,NaN,CCE1,33.48,-122.510
4,2008-11-11 15:17,15.352,33.317,349.4,382.6,386.0,NaN,NaN,NaN,NaN,CCE1,33.48,-122.510
...,...,...,...,...,...,...,...,...,...,...,...,...,...
490716,2021-10-22 21:17,29.985,33.325,472.9,390.8,407.1,NaN,NaN,NaN,NaN,La Parguera,17.95,-67.051
490717,2021-10-23 00:17,29.902,33.310,472.8,396.3,412.1,NaN,NaN,NaN,NaN,La Parguera,17.95,-67.051
490718,2021-10-23 03:17,29.876,33.335,477.9,396.1,411.8,NaN,NaN,NaN,NaN,La Parguera,17.95,-67.051
490719,2021-10-23 06:17,29.819,33.340,488.3,397.3,413.6,NaN,NaN,NaN,NaN,La Parguera,17.95,-67.051


In [178]:
df_LIAR = df_LIAR.drop(['CHL', 'NTU', 'pCO2_air', 'xCO2_air', 'DOXY', 'pH_sw'], axis=1)
df_LIAR

,datetime_utc,SST,SSS,pCO2_sw,site,latitude,longitude
0,2008-11-11 03:29,15.251,33.321,348.1,CCE1,33.48,-122.510
1,2008-11-11 06:17,15.207,33.326,351.0,CCE1,33.48,-122.510
2,2008-11-11 09:17,15.127,33.334,351.6,CCE1,33.48,-122.510
3,2008-11-11 12:17,15.075,33.350,351.7,CCE1,33.48,-122.510
4,2008-11-11 15:17,15.352,33.317,349.4,CCE1,33.48,-122.510
...,...,...,...,...,...,...,...
490716,2021-10-22 21:17,29.985,33.325,472.9,La Parguera,17.95,-67.051
490717,2021-10-23 00:17,29.902,33.310,472.8,La Parguera,17.95,-67.051
490718,2021-10-23 03:17,29.876,33.335,477.9,La Parguera,17.95,-67.051
490719,2021-10-23 06:17,29.819,33.340,488.3,La Parguera,17.95,-67.051


In [179]:
df_LIAR['depth'] = -0.5
df_LIAR

,datetime_utc,SST,SSS,pCO2_sw,site,latitude,longitude,depth
0,2008-11-11 03:29,15.251,33.321,348.1,CCE1,33.48,-122.510,-0.5
1,2008-11-11 06:17,15.207,33.326,351.0,CCE1,33.48,-122.510,-0.5
2,2008-11-11 09:17,15.127,33.334,351.6,CCE1,33.48,-122.510,-0.5
3,2008-11-11 12:17,15.075,33.350,351.7,CCE1,33.48,-122.510,-0.5
4,2008-11-11 15:17,15.352,33.317,349.4,CCE1,33.48,-122.510,-0.5
...,...,...,...,...,...,...,...,...
490716,2021-10-22 21:17,29.985,33.325,472.9,La Parguera,17.95,-67.051,-0.5
490717,2021-10-23 00:17,29.902,33.310,472.8,La Parguera,17.95,-67.051,-0.5
490718,2021-10-23 03:17,29.876,33.335,477.9,La Parguera,17.95,-67.051,-0.5
490719,2021-10-23 06:17,29.819,33.340,488.3,La Parguera,17.95,-67.051,-0.5


In [180]:
df_LIAR.to_csv('/Users/akbaskind/Desktop/YoE/buoy_LIAR.csv')

# Drop any rows where pH or pCO2 is `Nan`

While we originally start with 34 sites, many of the sites do not provide pCO2 or pH data. Here we remove any individual rows (site-independent) that have `Nan` pCO2 or pH. This results in many of the sites being entirely dropped, as they do not provide pCO2 or pH at any time step. This reduces our dataframe from 767670 rows with 34 sites to 130419 rows with 18 sites.

1. Select only rows from `df` where pH is not nan --> new dataframe `df1`
2. Select only rows from `df1` where pCO2_sw is not nan --> new dataframe `df2`

In [199]:
df1 = df[df['pH_sw'].notna()]
df2 = df1[df1['pCO2_sw'].notna()]
df2

,datetime_utc,SST,SSS,pCO2_sw,pCO2_air,xCO2_air,pH_sw,DOXY,CHL,NTU,site,latitude,longitude
9162,2013-10-03 12:17,16.944,33.468,372.9,388.2,393.9,8.071,231.3,NaN,NaN,CCE1,33.48,-122.510
9164,2013-10-03 18:17,16.225,33.535,380.5,388.6,393.7,8.069,232.2,NaN,NaN,CCE1,33.48,-122.510
9165,2013-10-03 21:17,16.408,33.574,381.8,388.5,394.1,8.065,230.7,NaN,NaN,CCE1,33.48,-122.510
9169,2013-10-04 09:17,16.429,33.333,372.1,387.9,394.5,8.079,237.0,0.733,NaN,CCE1,33.48,-122.510
9170,2013-10-04 12:17,16.187,33.374,374.7,387.2,393.8,8.073,237.1,NaN,NaN,CCE1,33.48,-122.510
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35407,2021-08-31 12:17,29.735,33.945,461.4,398.2,414.7,7.971,156.7,NaN,0.136,La Parguera,17.95,-67.051
35408,2021-08-31 15:17,30.265,33.961,457.9,391.8,408.4,7.983,182.5,NaN,0.095,La Parguera,17.95,-67.051
35409,2021-08-31 18:17,30.659,34.012,456.2,386.6,404.1,7.971,188.8,NaN,0.101,La Parguera,17.95,-67.051
35410,2021-08-31 21:17,30.652,34.015,454.6,387.6,405.3,7.977,190.9,NaN,0.114,La Parguera,17.95,-67.051


## Drop rows where `SSS` is `Nan`
Some sites at some time-steps do not provide sea surface salinity. Here we drop rows with `Nan` for `SSS`. This reduces our total dataframe size from 130419 rows to 130078 rows, so the data loss is small.

In [200]:
df2 = df2[df2['SSS'].notna()]
df2

,datetime_utc,SST,SSS,pCO2_sw,pCO2_air,xCO2_air,pH_sw,DOXY,CHL,NTU,site,latitude,longitude
9162,2013-10-03 12:17,16.944,33.468,372.9,388.2,393.9,8.071,231.3,NaN,NaN,CCE1,33.48,-122.510
9164,2013-10-03 18:17,16.225,33.535,380.5,388.6,393.7,8.069,232.2,NaN,NaN,CCE1,33.48,-122.510
9165,2013-10-03 21:17,16.408,33.574,381.8,388.5,394.1,8.065,230.7,NaN,NaN,CCE1,33.48,-122.510
9169,2013-10-04 09:17,16.429,33.333,372.1,387.9,394.5,8.079,237.0,0.733,NaN,CCE1,33.48,-122.510
9170,2013-10-04 12:17,16.187,33.374,374.7,387.2,393.8,8.073,237.1,NaN,NaN,CCE1,33.48,-122.510
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35407,2021-08-31 12:17,29.735,33.945,461.4,398.2,414.7,7.971,156.7,NaN,0.136,La Parguera,17.95,-67.051
35408,2021-08-31 15:17,30.265,33.961,457.9,391.8,408.4,7.983,182.5,NaN,0.095,La Parguera,17.95,-67.051
35409,2021-08-31 18:17,30.659,34.012,456.2,386.6,404.1,7.971,188.8,NaN,0.101,La Parguera,17.95,-67.051
35410,2021-08-31 21:17,30.652,34.015,454.6,387.6,405.3,7.977,190.9,NaN,0.114,La Parguera,17.95,-67.051


## Reset index of `df2` after dropping nan values

Since dropping `Nan` values is essentially applying a mask, we must reset the index of our dataframe to fully drop `Nan` values.

In [201]:
dff = df2.reset_index(drop=True)
dff

,datetime_utc,SST,SSS,pCO2_sw,pCO2_air,xCO2_air,pH_sw,DOXY,CHL,NTU,site,latitude,longitude
0,2013-10-03 12:17,16.944,33.468,372.9,388.2,393.9,8.071,231.3,NaN,NaN,CCE1,33.48,-122.510
1,2013-10-03 18:17,16.225,33.535,380.5,388.6,393.7,8.069,232.2,NaN,NaN,CCE1,33.48,-122.510
2,2013-10-03 21:17,16.408,33.574,381.8,388.5,394.1,8.065,230.7,NaN,NaN,CCE1,33.48,-122.510
3,2013-10-04 09:17,16.429,33.333,372.1,387.9,394.5,8.079,237.0,0.733,NaN,CCE1,33.48,-122.510
4,2013-10-04 12:17,16.187,33.374,374.7,387.2,393.8,8.073,237.1,NaN,NaN,CCE1,33.48,-122.510
...,...,...,...,...,...,...,...,...,...,...,...,...,...
130073,2021-08-31 12:17,29.735,33.945,461.4,398.2,414.7,7.971,156.7,NaN,0.136,La Parguera,17.95,-67.051
130074,2021-08-31 15:17,30.265,33.961,457.9,391.8,408.4,7.983,182.5,NaN,0.095,La Parguera,17.95,-67.051
130075,2021-08-31 18:17,30.659,34.012,456.2,386.6,404.1,7.971,188.8,NaN,0.101,La Parguera,17.95,-67.051
130076,2021-08-31 21:17,30.652,34.015,454.6,387.6,405.3,7.977,190.9,NaN,0.114,La Parguera,17.95,-67.051


## Load and reindex LIAR data

In [182]:
dfLIAR = pd.read_csv('/Users/akbaskind/Desktop/YoE/buoy_LIAR_out.csv')
dfLIAR = dfLIAR.drop(['index'], axis=1)
dfLIAR

,site,datetime,Latitude,Longitude,SSS,SST,pCO2_sw,Total Alkalinity [mmol/kg]
0,CCE1,2008-11-11 03:29,33.48,-122.510,33.321,15.251,348.1,2229.592272
1,CCE1,2008-11-11 06:17,33.48,-122.510,33.326,15.207,351.0,2229.876280
2,CCE1,2008-11-11 09:17,33.48,-122.510,33.334,15.127,351.6,2230.330831
3,CCE1,2008-11-11 12:17,33.48,-122.510,33.350,15.075,351.7,2231.238361
4,CCE1,2008-11-11 15:17,33.48,-122.510,33.317,15.352,349.4,2229.364108
...,...,...,...,...,...,...,...,...
490716,La Parguera,2021-10-22 21:17,17.95,-67.051,33.325,29.985,472.9,2204.381696
490717,La Parguera,2021-10-23 00:17,17.95,-67.051,33.310,29.902,472.8,2203.479760
490718,La Parguera,2021-10-23 03:17,17.95,-67.051,33.335,29.876,477.9,2204.937292
490719,La Parguera,2021-10-23 06:17,17.95,-67.051,33.340,29.819,488.3,2205.214395


# Round coordinates to match model coordinates

In [202]:
for i in dff.index:
    
    # Precise latitude
    lati = dff['latitude'][i]
    
    # Round latitude
    roundedlat = round(lati)
    
    # If latitude is rounded up, subtract half a latitude degree for the lat midpoint.
    # We know lat has been rounded up if the rounded lat is larger than the precise lat.
    if roundedlat > lati:
        finallat = roundedlat - 0.5
    # If latitude is rounded down, add half a degree.
    else:
        finallat = roundedlat + 0.5
        
    # Precise longitude
    longi = dff['longitude'][i]
    
    # Rounded longitude
    roundedlon = round(longi)
    
    # If longitude is rounded up, subtract half a longitude degree for the lon midpoint.
    # We know lon has been rounded up if the rounded lon is larger than the precise lon.
    if roundedlon > longi:
        finallon = roundedlon - 0.5
    # If longitude is rounded down, add half a degree.
    else:
        finallon = roundedlon + 0.5
        
    dff['latitude'][i] = finallat
    dff['longitude'][i] = finallon

## Round LIAR coords

In [183]:
for i in dfLIAR.index:
    
    # Precise latitude
    lati = dfLIAR['Latitude'][i]
    
    # Round latitude
    roundedlat = round(lati)
    
    # If latitude is rounded up, subtract half a latitude degree for the lat midpoint.
    # We know lat has been rounded up if the rounded lat is larger than the precise lat.
    if roundedlat > lati:
        finallat = roundedlat - 0.5
    # If latitude is rounded down, add half a degree.
    else:
        finallat = roundedlat + 0.5
        
    # Precise longitude
    longi = dfLIAR['Longitude'][i]
    
    # Rounded longitude
    roundedlon = round(longi)
    
    # If longitude is rounded up, subtract half a longitude degree for the lon midpoint.
    # We know lon has been rounded up if the rounded lon is larger than the precise lon.
    if roundedlon > longi:
        finallon = roundedlon - 0.5
    # If longitude is rounded down, add half a degree.
    else:
        finallon = roundedlon + 0.5
        
    dfLIAR['Latitude'][i] = finallat
    dfLIAR['Longitude'][i] = finallon

In [184]:
dfLIAR

,site,datetime,Latitude,Longitude,SSS,SST,pCO2_sw,Total Alkalinity [mmol/kg]
0,CCE1,2008-11-11 03:29,33.5,-122.5,33.321,15.251,348.1,2229.592272
1,CCE1,2008-11-11 06:17,33.5,-122.5,33.326,15.207,351.0,2229.876280
2,CCE1,2008-11-11 09:17,33.5,-122.5,33.334,15.127,351.6,2230.330831
3,CCE1,2008-11-11 12:17,33.5,-122.5,33.350,15.075,351.7,2231.238361
4,CCE1,2008-11-11 15:17,33.5,-122.5,33.317,15.352,349.4,2229.364108
...,...,...,...,...,...,...,...,...
490716,La Parguera,2021-10-22 21:17,17.5,-67.5,33.325,29.985,472.9,2204.381696
490717,La Parguera,2021-10-23 00:17,17.5,-67.5,33.310,29.902,472.8,2203.479760
490718,La Parguera,2021-10-23 03:17,17.5,-67.5,33.335,29.876,477.9,2204.937292
490719,La Parguera,2021-10-23 06:17,17.5,-67.5,33.340,29.819,488.3,2205.214395


# Drop metrics we do not care about
> chlorophyll

> NTU

> pCO2 air

> mixing ratio of air

> dissolved oxygen

Do this since Google sheets cannot accomodate more than 10,000,000 cells, and I've run into that problem several times.

In [203]:
dff = dff.drop(['CHL', 'NTU', 'pCO2_air', 'xCO2_air', 'DOXY'], axis=1)
# dff = dff.drop(['DOXY'], axis=1)

In [204]:
dff = dff.reset_index(drop=True)
dff

,datetime_utc,SST,SSS,pCO2_sw,pH_sw,site,latitude,longitude
0,2013-10-03 12:17,16.944,33.468,372.9,8.071,CCE1,33.5,-122.5
1,2013-10-03 18:17,16.225,33.535,380.5,8.069,CCE1,33.5,-122.5
2,2013-10-03 21:17,16.408,33.574,381.8,8.065,CCE1,33.5,-122.5
3,2013-10-04 09:17,16.429,33.333,372.1,8.079,CCE1,33.5,-122.5
4,2013-10-04 12:17,16.187,33.374,374.7,8.073,CCE1,33.5,-122.5
...,...,...,...,...,...,...,...,...
130073,2021-08-31 12:17,29.735,33.945,461.4,7.971,La Parguera,17.5,-67.5
130074,2021-08-31 15:17,30.265,33.961,457.9,7.983,La Parguera,17.5,-67.5
130075,2021-08-31 18:17,30.659,34.012,456.2,7.971,La Parguera,17.5,-67.5
130076,2021-08-31 21:17,30.652,34.015,454.6,7.977,La Parguera,17.5,-67.5


# Solve carb system for buoys to add `fCO2` and `OmegaAr`
INPUTS
> `pCO2_sw`: partial pressure of CO2 in seawater [uatm] -- type `4`

> `pH_sw`: seawater pH -- type `3`

> `SSS`: sea surface salinity

> `SST`: sea surface temperature [degC]

In [205]:
results = pyco2.sys(par1=dff['pCO2_sw'],par2=dff['pH_sw'],par1_type=4,par2_type=3,salinity = dff['SSS'], temperature = dff['SST'])
dff['fCO2'] = results['fCO2']
dff['OmegaAr'] = results['saturation_aragonite']

In [206]:
dff

,datetime_utc,SST,SSS,pCO2_sw,pH_sw,site,latitude,longitude,fCO2,OmegaAr
0,2013-10-03 12:17,16.944,33.468,372.9,8.071,CCE1,33.5,-122.5,371.586068,2.589660
1,2013-10-03 18:17,16.225,33.535,380.5,8.069,CCE1,33.5,-122.5,379.147099,2.552578
2,2013-10-03 21:17,16.408,33.574,381.8,8.065,CCE1,33.5,-122.5,380.445604,2.534092
3,2013-10-04 09:17,16.429,33.333,372.1,8.079,CCE1,33.5,-122.5,370.780363,2.622090
4,2013-10-04 12:17,16.187,33.374,374.7,8.073,CCE1,33.5,-122.5,373.367083,2.547543
...,...,...,...,...,...,...,...,...,...,...
130073,2021-08-31 12:17,29.735,33.945,461.4,7.971,La Parguera,17.5,-67.5,460.009967,3.115437
130074,2021-08-31 15:17,30.265,33.961,457.9,7.983,La Parguera,17.5,-67.5,456.529209,3.319253
130075,2021-08-31 18:17,30.659,34.012,456.2,7.971,La Parguera,17.5,-67.5,454.840694,3.168103
130076,2021-08-31 21:17,30.652,34.015,454.6,7.977,La Parguera,17.5,-67.5,453.245349,3.244982


In [186]:
resultsLIAR = pyco2.sys(par1=dfLIAR['pCO2_sw'],par2=dfLIAR['Total Alkalinity [mmol/kg]'],par1_type=4,par2_type=1,salinity = dfLIAR['SSS'], temperature = dfLIAR['SST'])
dfLIAR['fCO2'] = resultsLIAR['fCO2']
dfLIAR['OmegaAr'] = resultsLIAR['saturation_aragonite']
dfLIAR['pH_sw'] = resultsLIAR['pH']
dfLIAR

,site,datetime,Latitude,Longitude,SSS,SST,pCO2_sw,Total Alkalinity [mmol/kg],fCO2,OmegaAr,pH_sw
0,CCE1,2008-11-11 03:29,33.5,-122.5,33.321,15.251,348.1,2229.592272,346.846977,2.499994,8.092884
1,CCE1,2008-11-11 06:17,33.5,-122.5,33.326,15.207,351.0,2229.876280,349.735834,2.482486,8.089899
2,CCE1,2008-11-11 09:17,33.5,-122.5,33.334,15.127,351.6,2230.330831,350.332390,2.473719,8.089387
3,CCE1,2008-11-11 12:17,33.5,-122.5,33.350,15.075,351.7,2231.238361,350.431194,2.470876,8.089431
4,CCE1,2008-11-11 15:17,33.5,-122.5,33.317,15.352,349.4,2229.364108,348.143904,2.501720,8.091404
...,...,...,...,...,...,...,...,...,...,...,...
490716,La Parguera,2021-10-22 21:17,17.5,-67.5,33.325,29.985,472.9,2204.381696,471.479568,3.288538,7.978603
490717,La Parguera,2021-10-23 00:17,17.5,-67.5,33.310,29.902,472.8,2203.479760,471.378461,3.277476,7.978512
490718,La Parguera,2021-10-23 03:17,17.5,-67.5,33.335,29.876,477.9,2204.937292,476.462681,3.256467,7.974835
490719,La Parguera,2021-10-23 06:17,17.5,-67.5,33.340,29.819,488.3,2205.214395,486.830402,3.206699,7.967154


# Combine both buoy dataframes

In [207]:
dff = dff.rename(columns={"latitude": "Latitude", "longitude": "Longitude", "datetime_utc": "datetime"})
dff

,datetime,SST,SSS,pCO2_sw,pH_sw,site,Latitude,Longitude,fCO2,OmegaAr
0,2013-10-03 12:17,16.944,33.468,372.9,8.071,CCE1,33.5,-122.5,371.586068,2.589660
1,2013-10-03 18:17,16.225,33.535,380.5,8.069,CCE1,33.5,-122.5,379.147099,2.552578
2,2013-10-03 21:17,16.408,33.574,381.8,8.065,CCE1,33.5,-122.5,380.445604,2.534092
3,2013-10-04 09:17,16.429,33.333,372.1,8.079,CCE1,33.5,-122.5,370.780363,2.622090
4,2013-10-04 12:17,16.187,33.374,374.7,8.073,CCE1,33.5,-122.5,373.367083,2.547543
...,...,...,...,...,...,...,...,...,...,...
130073,2021-08-31 12:17,29.735,33.945,461.4,7.971,La Parguera,17.5,-67.5,460.009967,3.115437
130074,2021-08-31 15:17,30.265,33.961,457.9,7.983,La Parguera,17.5,-67.5,456.529209,3.319253
130075,2021-08-31 18:17,30.659,34.012,456.2,7.971,La Parguera,17.5,-67.5,454.840694,3.168103
130076,2021-08-31 21:17,30.652,34.015,454.6,7.977,La Parguera,17.5,-67.5,453.245349,3.244982


In [236]:
dff1 = dfLIAR[['site', 'datetime', 'Latitude', 'Longitude', 'SSS', 'SST', 'pCO2_sw', 'fCO2', 'OmegaAr', 'pH_sw']]
dff2 = dff[['site', 'datetime', 'Latitude', 'Longitude', 'SSS', 'SST', 'pCO2_sw', 'fCO2', 'OmegaAr', 'pH_sw']]
dff2

,site,datetime,Latitude,Longitude,SSS,SST,pCO2_sw,fCO2,OmegaAr,pH_sw
0,CCE1,2013-10-03 12:17,33.5,-122.5,33.468,16.944,372.9,371.586068,2.589660,8.071
1,CCE1,2013-10-03 18:17,33.5,-122.5,33.535,16.225,380.5,379.147099,2.552578,8.069
2,CCE1,2013-10-03 21:17,33.5,-122.5,33.574,16.408,381.8,380.445604,2.534092,8.065
3,CCE1,2013-10-04 09:17,33.5,-122.5,33.333,16.429,372.1,370.780363,2.622090,8.079
4,CCE1,2013-10-04 12:17,33.5,-122.5,33.374,16.187,374.7,373.367083,2.547543,8.073
...,...,...,...,...,...,...,...,...,...,...
130073,La Parguera,2021-08-31 12:17,17.5,-67.5,33.945,29.735,461.4,460.009967,3.115437,7.971
130074,La Parguera,2021-08-31 15:17,17.5,-67.5,33.961,30.265,457.9,456.529209,3.319253,7.983
130075,La Parguera,2021-08-31 18:17,17.5,-67.5,34.012,30.659,456.2,454.840694,3.168103,7.971
130076,La Parguera,2021-08-31 21:17,17.5,-67.5,34.015,30.652,454.6,453.245349,3.244982,7.977


In [237]:
resultdf = pd.concat([dff1, dff2])
len(resultdf['site'].unique())

34

In [238]:
resultdf = resultdf.reset_index(drop=True)
resultdf

,site,datetime,Latitude,Longitude,SSS,SST,pCO2_sw,fCO2,OmegaAr,pH_sw
0,CCE1,2008-11-11 03:29,33.5,-122.5,33.321,15.251,348.1,346.846977,2.499994,8.092884
1,CCE1,2008-11-11 06:17,33.5,-122.5,33.326,15.207,351.0,349.735834,2.482486,8.089899
2,CCE1,2008-11-11 09:17,33.5,-122.5,33.334,15.127,351.6,350.332390,2.473719,8.089387
3,CCE1,2008-11-11 12:17,33.5,-122.5,33.350,15.075,351.7,350.431194,2.470876,8.089431
4,CCE1,2008-11-11 15:17,33.5,-122.5,33.317,15.352,349.4,348.143904,2.501720,8.091404
...,...,...,...,...,...,...,...,...,...,...
620794,La Parguera,2021-08-31 12:17,17.5,-67.5,33.945,29.735,461.4,460.009967,3.115437,7.971000
620795,La Parguera,2021-08-31 15:17,17.5,-67.5,33.961,30.265,457.9,456.529209,3.319253,7.983000
620796,La Parguera,2021-08-31 18:17,17.5,-67.5,34.012,30.659,456.2,454.840694,3.168103,7.971000
620797,La Parguera,2021-08-31 21:17,17.5,-67.5,34.015,30.652,454.6,453.245349,3.244982,7.977000


In [239]:
resultdf.to_csv('/Users/akbaskind/Desktop/YoE/allbuoys.csv')

In [234]:
(1862397) > 10000000

False

In [13]:
%run buoydata_retrieval.py